In [1]:
from tensorflow.keras.models import load_model
model_name = '226_labels0.9608'
loaded_model = load_model(f"../models/Model_{model_name}.h5")

2025-05-02 12:10:36.325181: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-02 12:10:36.330354: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-02 12:10:36.344004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746168036.370337   31377 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746168036.377993   31377 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746168036.391864   31377 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
from keras.models import Model
from keras.layers import (Input, Bidirectional, LSTM, BatchNormalization, Dropout, 
                          MultiHeadAttention, Flatten, Dense, LeakyReLU, Conv1D, MaxPooling1D, GlobalAveragePooling1D)
from keras.regularizers import l2

input_shape = loaded_model.input_shape[1:]  # Get shape from loaded model
input_layer = Input(shape=input_shape)

#Convolution Layer
conv = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(input_layer)
conv = MaxPooling1D(pool_size=2)(conv)
# CPU-friendly LSTM (unroll=True, explicit activations)
lstm_out1 = Bidirectional(LSTM(256, return_sequences=True, activation='tanh',
                               recurrent_activation='sigmoid', unroll=True))(conv)
lstm_out1 = BatchNormalization()(lstm_out1)
lstm_out1 = Dropout(0.2)(lstm_out1)

lstm_out2 = Bidirectional(LSTM(256, return_sequences=True, activation='tanh',
                               recurrent_activation='sigmoid', unroll=True))(lstm_out1)
lstm_out2 = BatchNormalization()(lstm_out2)

lstm_out3 = Bidirectional(LSTM(256, return_sequences=True, activation='tanh',
                               recurrent_activation='sigmoid', unroll=True))(lstm_out2)
lstm_out3 = BatchNormalization()(lstm_out3)

attention_out = MultiHeadAttention(num_heads=4, key_dim=64)(lstm_out3, lstm_out3)

pooled=GlobalAveragePooling1D()(attention_out)

dense1 = Dense(128, kernel_regularizer=l2(1e-4))(pooled)
dense1 = LeakyReLU()(dense1)
dense1 = BatchNormalization()(dense1)
dense1 = Dropout(0.2)(dense1)

dense2 = Dense(64, kernel_regularizer=l2(1e-4))(dense1)
dense2 = LeakyReLU()(dense2)
dense2 = BatchNormalization()(dense2)
dense2 = Dropout(0.2)(dense2)

output_layer = Dense(loaded_model.output_shape[1], activation='softmax')(dense2)

model_cpu = Model(inputs=input_layer, outputs=output_layer)
model_cpu.set_weights(loaded_model.get_weights())

In [3]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model_cpu)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS
]
tflite_model = converter.convert()

with open(f"../models/{model_name}.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpqmvm_0_p/assets


INFO:tensorflow:Assets written to: /tmp/tmpqmvm_0_p/assets


Saved artifact at '/tmp/tmpqmvm_0_p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 543, 3), dtype=tf.float32, name='keras_tensor_41')
Output Type:
  TensorSpec(shape=(None, 226), dtype=tf.float32, name=None)
Captures:
  138633220641312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138633181174192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138634301923296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138633181175600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138633180862192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138633180872224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138633466684768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138634358905264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138633177787632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138633180868880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13863317777760

W0000 00:00:1746168071.465826   31377 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1746168071.465882   31377 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-05-02 12:11:11.467137: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpqmvm_0_p
2025-05-02 12:11:11.528317: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-05-02 12:11:11.528352: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpqmvm_0_p
I0000 00:00:1746168072.179095   31377 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-05-02 12:11:12.198160: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-05-02 12:11:13.531646: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpqmvm_0_p
2025-05-02 12:11:14.630334: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 